In [46]:
import pandas as pd
import cvxpy
import re

funds = [
    'AB Global High Yield AA HKD Inc.csv', 'CSOP Select US Dollar Bd A HKD.csv', 'E Fund (HK) HKD Mny Mkt B HKD A.csv', 'Harvest China A Research Select.csv',
    'AB SICAV I Low Volatility Eq A.csv', 'CUAM China-Hong Kong Strategy A.csv', 'E Fund (HK) Select Asia HY Bd A.csv', 'Harvest China Equity A HKD Acc.csv',
    'Allianz Global Artfcl Intlgc AT.csv', 'ChinaAMC Select Fixed Inc Allc.csv', 'E fund HK China equity.csv', 'Harvest China Income A HKD Inc.csv',
    'Allianz Income and Growth AM HK.csv', 'Da Cheng China Balanced A (HKD).csv', 'Franklin Biotechnology Discv A.csv', 'Janus Henderson Balanced A2 HKD.csv',
    'Allianz US Short Dur Hi Inc Bd.csv', 'Da Cheng Overseas China Concept.csv', 'Franklin Technology A Acc HKD.csv', 'Templeton Em Mkts Dyn Inc A MDi.csv',
    'Bosera Orient Sun Rise Grt CNH.csv', 'E Fund (HK) Grtr Chn Leaders A.csv', 'Franklin US Government A(acc)HK.csv',
]

In [49]:
df = pd.read_csv(f'./fund_data/{funds[0]}')
for fund in funds[1:]:
    _df = pd.read_csv(f'./fund_data/{fund}')
    df = df.merge(_df, how='left', left_on='Date', right_on='Date')
df.fillna(method='bfill', axis='index', inplace=True)
assets = df.columns[1:]
ll = df[assets[2]].tolist() 
ll = [float(l.replace(',', '')) for l in ll]
df[assets[2]] = ll
df

,Date,AB Global High Yield AA HKD Inc,CSOP Select US Dollar Bd A HKD Acc,E Fund (HK) HKD Mny Mkt B HKD Acc,Harvest China A Research Select A HKD,AB SICAV I Low Volatility Eq A HKD Acc,CUAM China-Hong Kong Strategy A,E Fund (HK) Select Asia HY Bd A HKD Inc,Harvest China Equity A HKD Acc,Allianz Global Artfcl Intlgc AT HKD,...,Da Cheng China Balanced A (HKD),Franklin Biotechnology Discv A(acc)HKD,Janus Henderson Balanced A2 HKD,Allianz US Short Dur Hi Inc Bd AM HKD,Da Cheng Overseas China Concept A HKD,Franklin Technology A Acc HKD,Templeton Em Mkts Dyn Inc A MDis HKD,Bosera Orient Sun Rise Grt CNH Bd A HKD,E Fund (HK) Grtr Chn Leaders A HKD Acc,Franklin US Government A(acc)HKD
0,5/6/14,117.13,10.00,1000.00,95.15,149.38,10.00,100.00,103.65,10.00,...,9.51,14.87,13.27,10.00,10.00,12.02,10.00,10.00,101.96,11.07
1,5/7/14,117.38,10.00,1000.00,93.89,149.94,10.00,100.00,101.59,10.00,...,9.47,14.71,13.32,10.00,10.00,11.95,10.00,10.00,101.96,11.08
2,5/8/14,117.51,10.00,1000.00,93.90,149.99,10.00,100.00,100.31,10.00,...,9.45,14.36,13.32,10.00,10.00,11.94,10.00,10.00,101.96,11.07
3,5/9/14,117.51,10.00,1000.00,93.03,149.99,10.00,100.00,99.36,10.00,...,9.38,14.64,13.31,10.00,10.00,11.97,10.00,10.00,101.96,11.06
4,5/10/14,117.51,10.00,1000.00,93.03,149.99,10.00,100.00,99.36,10.00,...,9.38,14.64,13.31,10.00,10.00,11.97,10.00,10.00,101.96,11.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2370,10/31/20,77.90,11.55,1027.66,152.86,225.34,21.79,92.83,225.04,19.13,...,14.63,25.24,19.95,9.02,19.68,43.73,8.26,11.09,105.23,12.13
2371,11/1/20,77.90,11.55,1027.66,152.86,225.34,21.79,92.83,225.04,19.13,...,14.63,25.24,19.95,9.02,19.68,43.73,8.26,11.09,105.23,12.13
2372,11/2/20,78.04,11.55,1027.69,154.70,227.93,21.79,92.85,227.16,18.80,...,14.73,25.23,20.09,9.05,19.88,43.74,8.31,11.10,106.14,12.14
2373,11/3/20,78.37,11.63,1027.70,156.17,232.62,21.81,93.12,228.22,18.88,...,14.90,25.78,20.33,9.06,19.90,44.26,8.32,11.11,106.79,12.14


In [50]:
Date = df['Date']
df.drop('Date', axis=1, inplace=True)
df

,AB Global High Yield AA HKD Inc,CSOP Select US Dollar Bd A HKD Acc,E Fund (HK) HKD Mny Mkt B HKD Acc,Harvest China A Research Select A HKD,AB SICAV I Low Volatility Eq A HKD Acc,CUAM China-Hong Kong Strategy A,E Fund (HK) Select Asia HY Bd A HKD Inc,Harvest China Equity A HKD Acc,Allianz Global Artfcl Intlgc AT HKD,ChinaAMC Select Fixed Inc Allc A HKD Acc,...,Da Cheng China Balanced A (HKD),Franklin Biotechnology Discv A(acc)HKD,Janus Henderson Balanced A2 HKD,Allianz US Short Dur Hi Inc Bd AM HKD,Da Cheng Overseas China Concept A HKD,Franklin Technology A Acc HKD,Templeton Em Mkts Dyn Inc A MDis HKD,Bosera Orient Sun Rise Grt CNH Bd A HKD,E Fund (HK) Grtr Chn Leaders A HKD Acc,Franklin US Government A(acc)HKD
0,117.13,10.00,1000.00,95.15,149.38,10.00,100.00,103.65,10.00,10.00,...,9.51,14.87,13.27,10.00,10.00,12.02,10.00,10.00,101.96,11.07
1,117.38,10.00,1000.00,93.89,149.94,10.00,100.00,101.59,10.00,10.00,...,9.47,14.71,13.32,10.00,10.00,11.95,10.00,10.00,101.96,11.08
2,117.51,10.00,1000.00,93.90,149.99,10.00,100.00,100.31,10.00,10.00,...,9.45,14.36,13.32,10.00,10.00,11.94,10.00,10.00,101.96,11.07
3,117.51,10.00,1000.00,93.03,149.99,10.00,100.00,99.36,10.00,10.00,...,9.38,14.64,13.31,10.00,10.00,11.97,10.00,10.00,101.96,11.06
4,117.51,10.00,1000.00,93.03,149.99,10.00,100.00,99.36,10.00,10.00,...,9.38,14.64,13.31,10.00,10.00,11.97,10.00,10.00,101.96,11.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2370,77.90,11.55,1027.66,152.86,225.34,21.79,92.83,225.04,19.13,10.55,...,14.63,25.24,19.95,9.02,19.68,43.73,8.26,11.09,105.23,12.13
2371,77.90,11.55,1027.66,152.86,225.34,21.79,92.83,225.04,19.13,10.55,...,14.63,25.24,19.95,9.02,19.68,43.73,8.26,11.09,105.23,12.13
2372,78.04,11.55,1027.69,154.70,227.93,21.79,92.85,227.16,18.80,10.55,...,14.73,25.23,20.09,9.05,19.88,43.74,8.31,11.10,106.14,12.14
2373,78.37,11.63,1027.70,156.17,232.62,21.81,93.12,228.22,18.88,10.55,...,14.90,25.78,20.33,9.06,19.90,44.26,8.32,11.11,106.79,12.14


In [52]:
df = pd.read_csv('demo_data.csv')
df

,Date,AB Global High Yield AA HKD Inc,CSOP Select US Dollar Bd A HKD Acc,E Fund (HK) HKD Mny Mkt B HKD Acc,Harvest China A Research Select A HKD,AB SICAV I Low Volatility Eq A HKD Acc,CUAM China-Hong Kong Strategy A,E Fund (HK) Select Asia HY Bd A HKD Inc,Harvest China Equity A HKD Acc,Allianz Global Artfcl Intlgc AT HKD,...,Da Cheng China Balanced A (HKD),Franklin Biotechnology Discv A(acc)HKD,Janus Henderson Balanced A2 HKD,Allianz US Short Dur Hi Inc Bd AM HKD,Da Cheng Overseas China Concept A HKD,Franklin Technology A Acc HKD,Templeton Em Mkts Dyn Inc A MDis HKD,Bosera Orient Sun Rise Grt CNH Bd A HKD,E Fund (HK) Grtr Chn Leaders A HKD Acc,Franklin US Government A(acc)HKD
0,5/6/14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5/7/14,0.002134,0.000000,0.000000,-0.013242,0.003749,0.000000,0.000000,-0.019875,0.000000,...,-0.004206,-0.010760,0.003768,0.000000,0.000000,-0.005824,0.000000,0.000000,0.000000,0.000903
2,5/8/14,0.001108,0.000000,0.000000,0.000107,0.000333,0.000000,0.000000,-0.012600,0.000000,...,-0.002112,-0.023793,0.000000,0.000000,0.000000,-0.000837,0.000000,0.000000,0.000000,-0.000903
3,5/9/14,0.000000,0.000000,0.000000,-0.009265,0.000000,0.000000,0.000000,-0.009471,0.000000,...,-0.007407,0.019499,-0.000751,0.000000,0.000000,0.002513,0.000000,0.000000,0.000000,-0.000903
4,5/10/14,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2370,10/31/20,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2371,11/1/20,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2372,11/2/20,0.001797,0.000000,0.000029,0.012037,0.011494,0.000000,0.000215,0.009421,-0.017250,...,0.006835,-0.000396,0.007018,0.003326,0.010163,0.000229,0.006053,0.000902,0.008648,0.000824
2373,11/3/20,0.004229,0.006926,0.000010,0.009502,0.020576,0.000918,0.002908,0.004666,0.004255,...,0.011541,0.021799,0.011946,0.001105,0.001006,0.011888,0.001203,0.000901,0.006124,0.000000


In [39]:
for asset in df.columns[1:]:
    df[asset] = df[asset].astype('float')
    df[asset] = df[asset].pct_change()
df

,Date,AB Global High Yield AA HKD Inc,CSOP Select US Dollar Bd A HKD Acc,E Fund (HK) HKD Mny Mkt B HKD Acc,Harvest China A Research Select A HKD,AB SICAV I Low Volatility Eq A HKD Acc,CUAM China-Hong Kong Strategy A,E Fund (HK) Select Asia HY Bd A HKD Inc,Harvest China Equity A HKD Acc,Allianz Global Artfcl Intlgc AT HKD,...,Da Cheng China Balanced A (HKD),Franklin Biotechnology Discv A(acc)HKD,Janus Henderson Balanced A2 HKD,Allianz US Short Dur Hi Inc Bd AM HKD,Da Cheng Overseas China Concept A HKD,Franklin Technology A Acc HKD,Templeton Em Mkts Dyn Inc A MDis HKD,Bosera Orient Sun Rise Grt CNH Bd A HKD,E Fund (HK) Grtr Chn Leaders A HKD Acc,Franklin US Government A(acc)HKD
0,5/6/14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5/7/14,0.002134,0.000000,0.000000,-0.013242,0.003749,0.000000,0.000000,-0.019875,0.000000,...,-0.004206,-0.010760,0.003768,0.000000,0.000000,-0.005824,0.000000,0.000000,0.000000,0.000903
2,5/8/14,0.001108,0.000000,0.000000,0.000107,0.000333,0.000000,0.000000,-0.012600,0.000000,...,-0.002112,-0.023793,0.000000,0.000000,0.000000,-0.000837,0.000000,0.000000,0.000000,-0.000903
3,5/9/14,0.000000,0.000000,0.000000,-0.009265,0.000000,0.000000,0.000000,-0.009471,0.000000,...,-0.007407,0.019499,-0.000751,0.000000,0.000000,0.002513,0.000000,0.000000,0.000000,-0.000903
4,5/10/14,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2370,10/31/20,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2371,11/1/20,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2372,11/2/20,0.001797,0.000000,0.000029,0.012037,0.011494,0.000000,0.000215,0.009421,-0.017250,...,0.006835,-0.000396,0.007018,0.003326,0.010163,0.000229,0.006053,0.000902,0.008648,0.000824
2373,11/3/20,0.004229,0.006926,0.000010,0.009502,0.020576,0.000918,0.002908,0.004666,0.004255,...,0.011541,0.021799,0.011946,0.001105,0.001006,0.011888,0.001203,0.000901,0.006124,0.000000


In [21]:
import riskfolio.Portfolio as pf

Y = 

# Building the portfolio object
port = pf.Portfolio(returns=Y)
# Calculating optimum portfolio

# Select method and estimate input parameters:

method_mu='hist' # Method to estimate expected returns based on historical data.
method_cov='hist' # Method to estimate covariance matrix based on historical data.

port.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.94)

# Estimate optimal portfolio:

model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
rm = 'MV' # Risk measure used, this time will be variance
obj = 'Sharpe' # Objective function, could be MinRisk, MaxRet, Utility or Sharpe
hist = True # Use historical scenarios for risk measures that depend on scenarios
rf = 0 # Risk free rate
l = 0 # Risk aversion factor, only useful when obj is 'Utility'

w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)

display(w.T)

ValueError: Shape of passed values is (1, 23), indices imply (1, 24)